In [3]:

import os
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tensorflow import keras

In [4]:
data_train = pd.read_csv('used_car/used_car_train_20200313.csv',sep = ' ')
data_test = pd.read_csv('used_car/used_car_testB_20200421.csv',sep = ' ')

In [5]:
data_train

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,149995,163978,20000607,121.0,10,4.0,0.0,1.0,163,15.0,...,0.280264,0.000310,0.048441,0.071158,0.019174,1.988114,-2.983973,0.589167,-1.304370,-0.302592
149996,149996,184535,20091102,116.0,11,0.0,0.0,0.0,125,10.0,...,0.253217,0.000777,0.084079,0.099681,0.079371,1.839166,-2.774615,2.553994,0.924196,-0.272160
149997,149997,147587,20101003,60.0,11,1.0,1.0,0.0,90,6.0,...,0.233353,0.000705,0.118872,0.100118,0.097914,2.439812,-1.630677,2.290197,1.891922,0.414931
149998,149998,45907,20060312,34.0,10,3.0,1.0,0.0,156,15.0,...,0.256369,0.000252,0.081479,0.083558,0.081498,2.075380,-2.633719,1.414937,0.431981,-1.659014


### 数据预处理

In [6]:
data_train.replace(to_replace = '-', value = np.nan, inplace = True)
data_test.replace(to_replace = '-', value = np.nan, inplace = True)
data_train.fillna(data_train.median(),inplace= True)
data_test.fillna(data_train.median(),inplace= True)

In [7]:
tags = ['model','brand','bodyType','fuelType','regionCode','regionCode','regDate','creatDate','kilometer','notRepairedDamage','power','v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']

In [8]:
data_train['power'][data_train['power']>600]=600
data_test['power'][data_test['power']>600]=600

<ipython-input-8-6e4c827fd50c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['power'][data_train['power']>600]=600
<ipython-input-8-6e4c827fd50c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['power'][data_test['power']>600]=600


In [9]:
# normalization
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data_train[tags].values)
x = min_max_scaler.transform(data_train[tags].values)
x_ = min_max_scaler.transform(data_test[tags].values)

### 模型预测

In [10]:
y = data_train['price'].values
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [11]:
model = keras.Sequential([
        keras.layers.Dense(250,activation='relu',input_shape=[26]), 
        keras.layers.Dense(250,activation='relu'), 
        keras.layers.Dense(250,activation='relu'), 
        keras.layers.Dense(1)])
model.compile(loss='mean_absolute_error', optimizer='Adam')

In [12]:
model.fit(x_train,y_train,batch_size = 2048,epochs=50)

Epoch 1/50
59/59 [==============================] - 3s 33ms/step - loss: 5836.2810
Epoch 2/50
59/59 [==============================] - 2s 32ms/step - loss: 4361.4165
Epoch 3/50
59/59 [==============================] - 2s 33ms/step - loss: 3905.1683
Epoch 4/50
59/59 [==============================] - 2s 31ms/step - loss: 2472.9919
Epoch 5/50
59/59 [==============================] - 2s 32ms/step - loss: 1417.0448
Epoch 6/50
59/59 [==============================] - 2s 33ms/step - loss: 1191.5575
Epoch 7/50
59/59 [==============================] - 2s 33ms/step - loss: 1103.8335
Epoch 8/50
59/59 [==============================] - 2s 32ms/step - loss: 1038.4132
Epoch 9/50
59/59 [==============================] - 2s 33ms/step - loss: 977.0889
Epoch 10/50
59/59 [==============================] - 2s 34ms/step - loss: 935.0731
Epoch 11/50
59/59 [==============================] - 2s 33ms/step - loss: 913.7559
Epoch 12/50
59/59 [==============================] - 2s 32ms/step - loss: 860.7389
Epoch

In [13]:
y1=model.predict(x_)

In [14]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10, max_depth=2)
model.fit(x_train,y_train)

RandomForestClassifier(max_depth=2, n_estimators=10)

In [15]:
y2 = model.predict(x_)

### 模型融合

In [24]:
y2 = y2.reshape([-1,1])

In [25]:
y2.shape

(50000, 1)

In [26]:
y_ = (y1 + y2) / 2

### 输出数据

In [27]:
data_test_price = pd.DataFrame(y_,columns = ['price'])
results = pd.concat([data_test['SaleID'],data_test_price],axis = 1)
results.to_csv('results.csv',sep = ',',index = None)